In [1]:
import pandas as pd
import numpy as np

In [2]:
def read_and_split(path, test_yr=20102011):
    """Read file and split into train and test sets. Use latest year
    for test set."""
    data = pd.read_csv(
        path, 
        header=0,
        names=['tract_2010', 'sch_year', 'grade', 'ct']
    )
    test = data[data['sch_year'] == test_yr]
    train = data[data['sch_year'] != test_yr]
    assert data.shape[0] == test.shape[0] + train.shape[0]
    return train, test

## Add some potential features.

* race
* population - static & growth
* age
* income
* household size 
* population age breakdown
* % change in students in past years

years: 2001 - 2009

In [60]:
def format_tract_for_match(driver):
    """Add 00 to end of each tract, except for tracts that refer to a decimal (where the second to last 
    character is 0).
    """
    driver['tract_formatted'] = driver.tract_2010.apply(
        lambda t: str(t) + '00' if str(t)[-2] != 0 else t)
    
def format_year(driver):
    driver['data_yr'] = driver.sch_year.apply(
        lambda y: str(y)[:4])
    
def join_pop_data(driver, data):
    joined = driver.merge(
        data, how='left', left_on=['tract_formatted', 'data_yr'], 
        right_on=['tract', 'yr']
    )
    return joined

In [76]:
# load pop data
pop = pd.read_csv(
        'data/New_York_City_Population_By_Census_Tracts.csv',
        names=['boro', 'yr', 'fips', 'dcp_boro_cd', 'tract', 'pop'],
        header=0)
bk_pop = pop[pop.boro == 'Brooklyn'].drop(['boro', 'fips', 'dcp_boro_cd'], axis=1)

bk_pop['yr'] = bk_pop['yr'].astype(str)
bk_pop['tract'] = bk_pop['tract'].astype(str)

# load driver data
train, test = read_and_split('data/driver/CSD20_Resident_Data_Phase_1.csv')
format_tract_for_match(train)
format_year(train)

# join
train2 = join_pop_data(train, bk_pop)
train2.head()

,tract_2010,sch_year,grade,ct,tract_formatted,data_yr,yr,tract,pop
0,18,20012002,2,1,1800,2001,NaN,NaN,NaN
1,18,20022003,3,1,1800,2002,NaN,NaN,NaN
2,18,20032004,K,1,1800,2003,NaN,NaN,NaN
3,18,20032004,3,1,1800,2003,NaN,NaN,NaN
4,18,20032004,4,1,1800,2003,NaN,NaN,NaN


In [78]:
train.dtypes

tract_2010          int64
sch_year            int64
grade              object
ct                  int64
tract_formatted    object
data_yr            object
dtype: object

In [81]:
print(bk_pop.dtypes)
bk_pop.head()

yr       object
tract    object
pop       int64
dtype: object


,yr,tract,pop
339,2000,100,5498
340,2000,200,1303
341,2000,301,5036
342,2000,501,3415
343,2000,502,2739
